In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]

In [6]:
df = df1.append(df2.append(df3))

In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
54,SEPTEMBER,25,Kajillionaire,Focus Features / Plan B Entertainment / Annapu...,Miranda July (director/screenplay); Evan Rache...,[145]
55,SEPTEMBER,25,The Last Shift,Stage 6 Films,Andrew Cohn (director/screenplay); Richard Jen...,[146]
56,SEPTEMBER,25,Secret Society of Second-Born Royals,Disney Channel / Disney+,"Anna Mastro (director); Alex Litvak, Andrew Gr...",[147]
57,SEPTEMBER,30,The Boys in the Band,Netflix,Joe Mantello (director); Mart Crowley (screenp...,[148]


In [8]:
df_2020 = df[['Title','Cast and crew']]

In [9]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
54,Kajillionaire,Miranda July (director/screenplay); Evan Rache...
55,The Last Shift,Andrew Cohn (director/screenplay); Richard Jen...
56,Secret Society of Second-Born Royals,"Anna Mastro (director); Alex Litvak, Andrew Gr..."
57,The Boys in the Band,Joe Mantello (director); Mart Crowley (screenp...


In [10]:
!pip install tmdbv3api

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable


In [11]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '3a1776ac024057ea0a9815e654f86763'

In [12]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [ ]:
df_2020['genre'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

In [14]:
df_2020

,Title,Cast and crew,genre
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery
...,...,...,...
49,Ma Rainey's Black Bottom,George C. Wolfe (director); Ruben Santiago-Hud...,Drama Music
50,The Father,Florian Zeller (director/screenplay); Christop...,Drama
51,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure
52,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western


In [15]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [16]:
df_2020['director'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_2020

,Title,Cast and crew,genre,director
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond
...,...,...,...,...
49,Ma Rainey's Black Bottom,George C. Wolfe (director); Ruben Santiago-Hud...,Drama Music,George C. Wolfe
50,The Father,Florian Zeller (director/screenplay); Christop...,Drama,Florian Zeller
51,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure,Patty Jenkins
52,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western,Paul Greengrass


In [18]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [19]:
df_2020['actors'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [21]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [22]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [23]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [24]:
df_2020['actors'] = df_2020['actors']+", "+df_2020['actor_2_name']+", "+df_2020['actor_3_name']

In [25]:
df_2020.drop(['actor_2_name','actor_3_name'],axis=1,inplace=True)

In [26]:
df_2020

,Title,Cast and crew,genre,director,actors
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,"Andrea Riseborough, Demián Bichir, John Cho"
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,"Kristen Stewart, Vincent Cassel, Jessica Henwick"
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,"Tiffany Haddish, Rose Byrne, Salma Hayek"
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,"Josh Hartnett, Margarita Levieva, Chandler Riggs"
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,"Freya Tingley, Simon Abkarian, Rutger Hauer"
...,...,...,...,...,...
49,Ma Rainey's Black Bottom,George C. Wolfe (director); Ruben Santiago-Hud...,Drama Music,George C. Wolfe,"Viola Davis, Chadwick Boseman, Glynn Turman"
50,The Father,Florian Zeller (director/screenplay); Christop...,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss"
51,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure,Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig"
52,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western,Paul Greengrass,"Tom Hanks, Helena Zengel, Neil Sandilands"


In [27]:
df_2020 = df_2020.rename(columns={'Title':'title'})

In [28]:
df_2020

,title,Cast and crew,genre,director,actors
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,"Andrea Riseborough, Demián Bichir, John Cho"
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,"Kristen Stewart, Vincent Cassel, Jessica Henwick"
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,"Tiffany Haddish, Rose Byrne, Salma Hayek"
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,"Josh Hartnett, Margarita Levieva, Chandler Riggs"
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,"Freya Tingley, Simon Abkarian, Rutger Hauer"
...,...,...,...,...,...
49,Ma Rainey's Black Bottom,George C. Wolfe (director); Ruben Santiago-Hud...,Drama Music,George C. Wolfe,"Viola Davis, Chadwick Boseman, Glynn Turman"
50,The Father,Florian Zeller (director/screenplay); Christop...,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss"
51,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure,Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig"
52,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western,Paul Greengrass,"Tom Hanks, Helena Zengel, Neil Sandilands"


In [29]:
new_df20 = df_2020.loc[:,['title','genre','director','actors']]

In [30]:
new_df20

,title,genre,director,actors
0,The Grudge,Horror Mystery,Nicolas Pesce,"Andrea Riseborough, Demián Bichir, John Cho"
1,Underwater,Action Horror Science Fiction Thriller,William Eubank,"Kristen Stewart, Vincent Cassel, Jessica Henwick"
2,Like a Boss,Comedy,Miguel Arteta,"Tiffany Haddish, Rose Byrne, Salma Hayek"
3,Inherit the Viper,Drama Thriller Crime,Anthony Jerjen,"Josh Hartnett, Margarita Levieva, Chandler Riggs"
4,The Sonata,Horror Thriller Mystery,Andrew Desmond,"Freya Tingley, Simon Abkarian, Rutger Hauer"
...,...,...,...,...
49,Ma Rainey's Black Bottom,Drama Music,George C. Wolfe,"Viola Davis, Chadwick Boseman, Glynn Turman"
50,The Father,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss"
51,Wonder Woman 1984,Fantasy Action Adventure,Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig"
52,News of the World,Drama Western,Paul Greengrass,"Tom Hanks, Helena Zengel, Neil Sandilands"


In [31]:
new_df20['comb'] = new_df20['actors'] + ' '+ new_df20['director'] +' ' + new_df20['genre']

In [32]:
new_df20 = new_df20.dropna(how='any')

In [33]:
new_df20

,title,genre,director,actors,comb
0,The Grudge,Horror Mystery,Nicolas Pesce,"Andrea Riseborough, Demián Bichir, John Cho","Andrea Riseborough, Demián Bichir, John Cho Ni..."
1,Underwater,Action Horror Science Fiction Thriller,William Eubank,"Kristen Stewart, Vincent Cassel, Jessica Henwick","Kristen Stewart, Vincent Cassel, Jessica Henwi..."
2,Like a Boss,Comedy,Miguel Arteta,"Tiffany Haddish, Rose Byrne, Salma Hayek","Tiffany Haddish, Rose Byrne, Salma Hayek Migue..."
3,Inherit the Viper,Drama Thriller Crime,Anthony Jerjen,"Josh Hartnett, Margarita Levieva, Chandler Riggs","Josh Hartnett, Margarita Levieva, Chandler Rig..."
4,The Sonata,Horror Thriller Mystery,Andrew Desmond,"Freya Tingley, Simon Abkarian, Rutger Hauer","Freya Tingley, Simon Abkarian, Rutger Hauer An..."
...,...,...,...,...,...
49,Ma Rainey's Black Bottom,Drama Music,George C. Wolfe,"Viola Davis, Chadwick Boseman, Glynn Turman","Viola Davis, Chadwick Boseman, Glynn Turman Ge..."
50,The Father,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss","Anthony Hopkins, Olivia Colman, Mark Gatiss Fl..."
51,Wonder Woman 1984,Fantasy Action Adventure,Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig","Gal Gadot, Chris Pine, Kristen Wiig Patty Jenk..."
52,News of the World,Drama Western,Paul Greengrass,"Tom Hanks, Helena Zengel, Neil Sandilands","Tom Hanks, Helena Zengel, Neil Sandilands Paul..."


In [34]:
old_df = pd.read_csv('data_prep1.csv')

In [35]:
old_df

,imdb_title_id,title,genre,director,actors,comb
0,tt0000574,The Story of the Kelly Gang,"Biography, Crime, Drama",Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",Elizabeth Tait John Tait Norman Campbell Be...
1,tt0001892,Den sorte drøm,Drama,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Asta Nielsen Valdemar Psilander Gunnar Helse...
2,tt0002101,Cleopatra,"Drama, History",Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",Helen Gardner Pearl Sindelar Miss Fielding ...
3,tt0002130,L'Inferno,"Adventure, Drama, Fantasy","Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Salvatore Papa Arturo Pirovano Giuseppe de L...
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",Sidney Olcott,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...",R. Henderson Bland Percy Dyer Gene Gauntier ...
...,...,...,...,...,...,...
81268,tt9903716,Jessie,"Horror, Thriller",Aswani Kumar V.,"Sritha Chandana, Pavani Gangireddy, Abhinav Go...",Sritha Chandana Pavani Gangireddy Abhinav Go...
81269,tt9905412,Ottam,Drama,Zam,"Nandu Anand, Roshan Ullas, Manikandan R. Achar...",Nandu Anand Roshan Ullas Manikandan R. Achar...
81270,tt9905462,Pengalila,Drama,T.V. Chandran,"Lal, Akshara Kishor, Iniya, Narain, Renji Pani...",Lal Akshara Kishor Iniya Narain Renji Pani...
81271,tt9911774,Padmavyuhathile Abhimanyu,Drama,Vineesh Aaradya,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",Anoop Chandran Indrans Sona Nair Simon Brit...


In [36]:
final_df = old_df.append(new_df20,ignore_index=True)

In [37]:
final_df 

,imdb_title_id,title,genre,director,actors,comb
0,tt0000574,The Story of the Kelly Gang,"Biography, Crime, Drama",Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",Elizabeth Tait John Tait Norman Campbell Be...
1,tt0001892,Den sorte drøm,Drama,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Asta Nielsen Valdemar Psilander Gunnar Helse...
2,tt0002101,Cleopatra,"Drama, History",Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",Helen Gardner Pearl Sindelar Miss Fielding ...
3,tt0002130,L'Inferno,"Adventure, Drama, Fantasy","Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Salvatore Papa Arturo Pirovano Giuseppe de L...
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",Sidney Olcott,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...",R. Henderson Bland Percy Dyer Gene Gauntier ...
...,...,...,...,...,...,...
81400,NaN,Ma Rainey's Black Bottom,Drama Music,George C. Wolfe,"Viola Davis, Chadwick Boseman, Glynn Turman","Viola Davis, Chadwick Boseman, Glynn Turman Ge..."
81401,NaN,The Father,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss","Anthony Hopkins, Olivia Colman, Mark Gatiss Fl..."
81402,NaN,Wonder Woman 1984,Fantasy Action Adventure,Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig","Gal Gadot, Chris Pine, Kristen Wiig Patty Jenk..."
81403,NaN,News of the World,Drama Western,Paul Greengrass,"Tom Hanks, Helena Zengel, Neil Sandilands","Tom Hanks, Helena Zengel, Neil Sandilands Paul..."


In [38]:
final_df.drop(['imdb_title_id'],axis=1,inplace=True)

In [42]:
final_df['title'] = final_df['title'].str.lower()

In [43]:
final_df

,title,genre,director,actors,comb
0,the story of the kelly gang,"Biography, Crime, Drama",Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",Elizabeth Tait John Tait Norman Campbell Be...
1,den sorte drøm,Drama,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Asta Nielsen Valdemar Psilander Gunnar Helse...
2,cleopatra,"Drama, History",Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",Helen Gardner Pearl Sindelar Miss Fielding ...
3,l'inferno,"Adventure, Drama, Fantasy","Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Salvatore Papa Arturo Pirovano Giuseppe de L...
4,"from the manger to the cross; or, jesus of naz...","Biography, Drama",Sidney Olcott,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...",R. Henderson Bland Percy Dyer Gene Gauntier ...
...,...,...,...,...,...
81400,ma rainey's black bottom,Drama Music,George C. Wolfe,"Viola Davis, Chadwick Boseman, Glynn Turman","Viola Davis, Chadwick Boseman, Glynn Turman Ge..."
81401,the father,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss","Anthony Hopkins, Olivia Colman, Mark Gatiss Fl..."
81402,wonder woman 1984,Fantasy Action Adventure,Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig","Gal Gadot, Chris Pine, Kristen Wiig Patty Jenk..."
81403,news of the world,Drama Western,Paul Greengrass,"Tom Hanks, Helena Zengel, Neil Sandilands","Tom Hanks, Helena Zengel, Neil Sandilands Paul..."


In [44]:
final_df.to_csv('main_data.csv', index=False)